# Recommending Without User Data: Content Filtering With NLP and Cosine Similarity

Using data previously cleaned, I'm going to use several methods of NLP along with cosine similarity to make a simple, initial movie recommendation system. This sytem can be used to recommend movies when users are brand new. There are several ways this can be used:

1. The user can search for similar movies to a movie they like when a user first signs up with the app or service, but movies will be very closely related (Iron Man, Iron Man II, Iron Man III, etc)
2. The app can use this feature to suggest similar movies for the User to initially rate. I.e if the user says they like Toy Story, movies with a high cosine similarity can be the next suggestions for a user to rate the movie. 
3. This method can be used to group similar movies together into recommended "pods". For example, "Japanese animation featuring strong female leads" might include Spirited Away and Princess Mononoke.


### Features Used For Recommendations

* **Rated** - The movie's rating
* **Year** - The Release Year
* **Genre** - The movie's genres
* **Writer** - Who wrote the screenplay
* **Actors** - Significant actors in the movie
* **Director** - who directed the movies
* **Key words** - Key words taken from the plot using [nltk_rake](https://pypi.org/project/rake-nltk/)

In [1]:
import pandas as pd
import numpy as np

import nltk
from rake_nltk import Rake
from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import time 

# 1. Bag of Words and TF-IDF

In [2]:
movies = pd.read_csv('/Users/user/final-capstone/inprogress3.csv')
print(movies.shape)
movies.head()

/Users/user/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(284303, 25)


,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,metascore,imdb_rating,imdb_votes,imdb_id,type,dvd,box_office,production,website,response
0,Toy Story,1995,G,22-Nov-95,81 min,"Animation, Adventure, Comedy, Family, Fantasy",John Lasseter,"John Lasseter (original story by), Pete Docter...","Tom Hanks, Tim Allen, Don Rickles, Jim Varney",A cowboy doll is profoundly threatened and jea...,...,95.0,8.3,"788,709",tt0114709,movie,20-Mar-01,NaN,Buena Vista,http://www.disney.com/ToyStory,True
1,Jumanji,1995,PG,15-Dec-95,104 min,"Adventure, Comedy, Family, Fantasy",Joe Johnston,"Jonathan Hensleigh (screenplay by), Greg Taylo...","Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",When two kids find and play a magical board ga...,...,39.0,7.0,"267,973",tt0113497,movie,25-Jan-00,NaN,Sony Pictures Home Entertainment,NaN,True
2,Grumpier Old Men,1995,PG-13,22-Dec-95,101 min,"Comedy, Romance",Howard Deutch,"Mark Steven Johnson (characters), Mark Steven ...","Walter Matthau, Jack Lemmon, Sophia Loren, Ann...",John and Max resolve to save their beloved bai...,...,46.0,6.6,"22,329",tt0113228,movie,18-Nov-97,NaN,Warner Home Video,NaN,True
3,Waiting to Exhale,1995,R,22-Dec-95,124 min,"Comedy, Drama, Romance",Forest Whitaker,"Terry McMillan (novel), Terry McMillan (screen...","Whitney Houston, Angela Bassett, Loretta Devin...","Based on Terry McMillan's novel, this film fol...",...,NaN,5.8,"8,640",tt0114885,movie,6-Mar-01,NaN,Twentieth Century Fox Home Entertainment,NaN,True
4,Father of the Bride Part II,1995,PG,8-Dec-95,106 min,"Comedy, Family, Romance",Charles Shyer,"Albert Hackett (screenplay ""Father's Little Di...","Steve Martin, Diane Keaton, Martin Short, Kimb...",George Banks must deal not only with the pregn...,...,49.0,6.0,"31,241",tt0113041,movie,9-May-00,NaN,Disney,NaN,True


In [3]:
movies = movies.drop(['website','response','type','ratings','metascore','dvd','box_office'],axis=1)


# Drop TV shows from the dataframe
ratings = ['TV-14', 'TV-PG', 'TV-MA', 'TV-G', 'TV-Y', 'TV-13', 'TV-Y7']

for rating in ratings:
    movies = movies[movies.rated != rating]
    
    
# Drop a few other misc TV shows that snuck through
titles = ['The Mahabharata', 'Centennial', 'Dominion Tank Police', 'Children of Dune', 'Helen of Troy']

for title in titles:
    movies = movies[movies['title'] != title]

# putting together the year with the movie title
movies['year'] = movies['year'].apply(lambda x: str(x))
movies['title_year'] = movies['title'].map(str) + ' (' + movies['year'] + ')'
    

movies['year'] = movies['year'].str.strip('–')
movies = movies[movies['year'] != 'nan']
movies['year'] = pd.to_numeric(movies['year'])
movies['year'] = movies.year.astype(int)

# Dropping movies that have fewer than 100 ratings on imdb only so that we can have computational power

movies['imdb_votes'] = movies['imdb_votes'].str.replace(",","")
movies['imdb_votes'] = pd.to_numeric(movies['imdb_votes'])
print(movies.shape)

(274600, 19)


In [4]:
vote_counts = movies[movies['imdb_votes'].notnull()]['imdb_votes'].astype('int')
vote_averages = movies[movies['imdb_rating'].notnull()]['imdb_rating'].astype('int')
C = vote_averages.mean()

m = vote_counts.quantile(0.75)

def weighted_rating(x):
    v = x['imdb_votes']+1 # added +1 - Dan
    R = x['imdb_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

movies['weighted_rating'] = movies.apply(weighted_rating, axis=1)

In [11]:
# movies.to_csv('scraped-movies-with-wr-ty.csv', index=False)

In [5]:
movies.columns

Index(['title', 'year', 'rated', 'released', 'runtime', 'genre', 'director',
       'writer', 'actors', 'plot', 'language', 'country', 'awards', 'poster',
       'imdb_rating', 'imdb_votes', 'imdb_id', 'production', 'title_year',
       'weighted_rating'],
      dtype='object')

For computational reasons, we're going to pare down the data a bit. So far I've found that I can only scale the cosine similarity to about 51,000 movies. I do want to include popular movies, so I'm going to only include movies with over 100 IMDB votes and see how that does.

In [8]:
# Cosine Similarity Paring down - Want below 52,000 movies

threshold_votes = movies['imdb_votes'].quantile(.58)
quarter = movies['imdb_rating'].quantile(.36)

cos_movies = movies[(movies['imdb_votes'] >= threshold_votes) & (movies['imdb_rating'] >= quarter)
                    & (movies['year'] >= 1950) ]

cos_movies.shape

(50794, 20)

In [9]:
cos_movies = cos_movies[['title_year','weighted_rating','imdb_rating','imdb_id','rated','writer','genre','director','actors','plot','language','country', 'year']]

for column in cos_movies:
    cos_movies[column] = cos_movies[column].apply(lambda x: str(x))
    
# discarding the commas between the actors' full names
cos_movies['actors'] = cos_movies['actors'].map(lambda x: x.split(','))

# putting the genres in a list of words
cos_movies['genre'] = cos_movies['genre'].map(lambda x: x.lower().split(','))

cos_movies['director'] = cos_movies['director'].map(lambda x: x.split(','))


# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in cos_movies.iterrows():
    row['actors'] = [x.lower().replace(' ','') for x in row['actors']]
    row['director'] = [x.lower().replace(' ','') for x in row['director']]


# Getting rid of information between () in writers
cos_movies['writer'] = cos_movies['writer'].str.replace(r"\(.*\)","")

# discarding the commas between writer's names
cos_movies['writer'] = cos_movies['writer'].map(lambda x: x.split(','))

# putting the countries in a list of words
cos_movies['country'] = cos_movies['country'].map(lambda x: x.lower().split(','))

# putting the languages in a list of words
cos_movies['language'] = cos_movies['language'].map(lambda x: x.lower().split(','))

cos_movies['rated'] = cos_movies['rated'].str.replace("-","")

cos_movies['year'] = cos_movies['year'].map(lambda x: x.split(','))

# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in cos_movies.iterrows():
    row['writer'] = [x.lower().replace(' ','') for x in row['writer']]

cos_movies.head()

,title_year,weighted_rating,imdb_rating,imdb_id,rated,writer,genre,director,actors,plot,language,country,year
0,Toy Story (1995),8.29874801320629,8.3,tt0114709,G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",A cowboy doll is profoundly threatened and jea...,[english],[usa],[1995]
1,Jumanji (1995),6.9981594078752325,7.0,tt0113497,PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...",When two kids find and play a magical board ga...,"[english, french]",[usa],[1995]
2,Grumpier Old Men (1995),6.584943625757381,6.6,tt0113228,PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...",John and Max resolve to save their beloved bai...,"[english, italian, german]",[usa],[1995]
3,Waiting to Exhale (1995),5.795795337651051,5.8,tt0114885,R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...","Based on Terry McMillan's novel, this film fol...",[english],[usa],[1995]
4,Father of the Bride Part II (1995),5.99639712038929,6.0,tt0113041,PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",George Banks must deal not only with the pregn...,[english],[usa],[1995]


In [10]:
cos_movies = cos_movies.drop(['weighted_rating','imdb_id','imdb_rating'],axis=1)
cos_movies.columns

Index(['title_year', 'rated', 'writer', 'genre', 'director', 'actors', 'plot',
       'language', 'country', 'year'],
      dtype='object')

In [11]:
# initializing the new column
cos_movies['Key_words'] = ""

for index, row in cos_movies.iterrows():
    plot = row['plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
cos_movies.drop(columns = ['plot'], inplace = True)

In [12]:
cos_movies.head()

,title_year,rated,writer,genre,director,actors,language,country,year,Key_words
0,Toy Story (1995),G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",[english],[usa],[1995],"[profoundly, threatened, room, new, spaceman, ..."
1,Jumanji (1995),PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...","[english, french]",[usa],[1995],"[magical, board, game, host, man, trapped, dan..."
2,Grumpier Old Men (1995),PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...","[english, italian, german]",[usa],[1995],"[turning, john, max, resolve, save, beloved, b..."
3,Waiting to Exhale (1995),R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...",[english],[usa],[1995],"[relationships, american, women, film, follows..."
4,Father of the Bride Part II (1995),PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",[english],[usa],[1995],"[daughter, pregnancy, also, wife, george, bank..."


In [13]:
cos_movies.drop_duplicates(subset='title_year',keep='first')
cos_movies.shape

(50794, 10)

In [14]:
cos_movies.set_index('title_year', inplace = True)
cos_movies.head()

,rated,writer,genre,director,actors,language,country,year,Key_words
title_year,,,,,,,,,
Toy Story (1995),G,[johnlasseter],"[animation, adventure, comedy, family, fan...",[johnlasseter],"[tomhanks, timallen, donrickles, jimvarney]",[english],[usa],[1995],"[profoundly, threatened, room, new, spaceman, ..."
Jumanji (1995),PG,[jonathanhensleigh],"[adventure, comedy, family, fantasy]",[joejohnston],"[robinwilliams, jonathanhyde, kirstendunst, br...","[english, french]",[usa],[1995],"[magical, board, game, host, man, trapped, dan..."
Grumpier Old Men (1995),PG13,"[markstevenjohnson, markstevenjohnson]","[comedy, romance]",[howarddeutch],"[waltermatthau, jacklemmon, sophialoren, ann-m...","[english, italian, german]",[usa],[1995],"[turning, john, max, resolve, save, beloved, b..."
Waiting to Exhale (1995),R,[terrymcmillan],"[comedy, drama, romance]",[forestwhitaker],"[whitneyhouston, angelabassett, lorettadevine,...",[english],[usa],[1995],"[relationships, american, women, film, follows..."
Father of the Bride Part II (1995),PG,[alberthackett],"[comedy, family, romance]",[charlesshyer],"[stevemartin, dianekeaton, martinshort, kimber...",[english],[usa],[1995],"[daughter, pregnancy, also, wife, george, bank..."


In [15]:
cos_movies['bag_of_words'] = ''
columns = cos_movies.columns
for index, row in cos_movies.iterrows():
    words = ''
    for col in columns:
        if col == 'rated':
            words = words.join(row[col]) + ' '
        elif col == 'imdb_rating':
            None
        else:
            words = words + ' '.join(row[col])+ ' '

    row['bag_of_words'] = words
    
cos_movies.drop(columns = [col for col in cos_movies.columns if col!= 'bag_of_words'], inplace = True)

In [16]:
cos_movies.head()

,bag_of_words
title_year,
Toy Story (1995),G johnlasseter animation adventure comedy f...
Jumanji (1995),PG jonathanhensleigh adventure comedy family...
Grumpier Old Men (1995),PG13 markstevenjohnson markstevenjohnson comed...
Waiting to Exhale (1995),R terrymcmillan comedy drama romance forestw...
Father of the Bride Part II (1995),PG alberthackett comedy family romance charl...


In [17]:
cos_movies['bag_of_words'][0]

'G johnlasseter animation  adventure  comedy  family  fantasy johnlasseter tomhanks timallen donrickles jimvarney english usa 1995 profoundly threatened room new spaceman figure supplants cowboy doll jealous top toy boy  '

In [18]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(cos_movies['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(cos_movies.index)
indices[:5]

0                      Toy Story (1995)
1                        Jumanji (1995)
2               Grumpier Old Men (1995)
3              Waiting to Exhale (1995)
4    Father of the Bride Part II (1995)
Name: title_year, dtype: object

In [19]:
times = []

start = time.time()

cosine_sim = cosine_similarity(count_matrix, count_matrix)
cos_sim_time = time.time() - start
times.append(cos_sim_time)
print("Runtime %0.2f" % (time.time() - start))
print(cosine_sim)

Runtime 368.37
[[1.         0.23732223 0.15694121 ... 0.         0.06896552 0.03052813]
 [0.23732223 1.         0.12344268 ... 0.         0.06780635 0.06003002]
 [0.15694121 0.12344268 1.         ... 0.         0.03138824 0.055577  ]
 ...
 [0.         0.         0.         ... 1.         0.09925833 0.08787496]
 [0.06896552 0.06780635 0.03138824 ... 0.09925833 1.         0.06105625]
 [0.03052813 0.06003002 0.055577   ... 0.08787496 0.06105625 1.        ]]


In [20]:
cosine_sim.shape

(50794, 50794)

In [21]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_15_indexes = list(score_series.iloc[1:16].index)
    
    # populating the list with the titles of the best 15 matching movies
    for i in top_15_indexes:
        recommended_movies.append(list(cos_movies.index)[i])
        
    return recommended_movies

In [22]:
def return_recs(title, orderby, df):
    title_a = recommendations(title)
    title_recs = df[df.index.isin(title_a)]
    print("Here are the movie recommendations for {}: ".format(title))
    if orderby == None:
        return title_recs[:10]
    else:
        return title_recs[:10].sort_values(by=[orderby], ascending=False)

In [28]:
movies = pd.read_csv('scraped-movies-with-wr-ty.csv')
movies = movies.drop('Unnamed: 0', axis=1)
movies = movies.drop_duplicates(subset='title_year', keep='first')
movies = movies.set_index('title_year')

cols = ['rated','plot','year','genre','director','actors','language','weighted_rating']
movies = movies[cols]
movies.head()


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Toy Story (1995),G,A cowboy doll is profoundly threatened and jea...,1995,"Animation, Adventure, Comedy, Family, Fantasy",John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",English,8.298748
Jumanji (1995),PG,When two kids find and play a magical board ga...,1995,"Adventure, Comedy, Family, Fantasy",Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","English, French",6.998159
Grumpier Old Men (1995),PG-13,John and Max resolve to save their beloved bai...,1995,"Comedy, Romance",Howard Deutch,"Walter Matthau, Jack Lemmon, Sophia Loren, Ann...","English, Italian, German",6.584944
Waiting to Exhale (1995),R,"Based on Terry McMillan's novel, this film fol...",1995,"Comedy, Drama, Romance",Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...",English,5.795795
Father of the Bride Part II (1995),PG,George Banks must deal not only with the pregn...,1995,"Comedy, Family, Romance",Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",English,5.996397


In [29]:
return_recs('Zootopia (2016)', 'weighted_rating', df = movies)

Here are the movie recommendations for Zootopia (2016): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
"Monsters, Inc. (2001)",G,"In order to power the city, monsters have to s...",2001,"Animation, Adventure, Comedy, Family, Fantasy","Pete Docter, David Silverman(co-director), Lee...","John Goodman, Billy Crystal, Mary Gibbs, Steve...",English,8.098755
Fantastic Mr. Fox (2009),PG,An urbane fox cannot resist returning to his f...,2009,"Animation, Adventure, Comedy, Crime, Drama, Fa...",Wes Anderson,"George Clooney, Meryl Streep, Jason Schwartzma...","English, French",7.895249
Who Framed Roger Rabbit (1988),PG,A toon-hating detective is a cartoon rabbit's ...,1988,"Animation, Adventure, Comedy, Crime, Family, F...",Robert Zemeckis,"Bob Hoskins, Christopher Lloyd, Joanna Cassidy...",English,7.695482
Sing (2016),PG,"In a city of humanoid animals, a hustling thea...",2016,"Animation, Comedy, Family, Musical","Garth Jennings, Christophe Lourdelet(co-director)","Matthew McConaughey, Reese Witherspoon, Seth M...","English, Japanese, Ukrainian",7.095406
Bolt (2008),PG,The canine star of a fictional sci-fi/action s...,2008,"Animation, Adventure, Comedy, Drama, Family","Byron Howard, Chris Williams","John Travolta, Miley Cyrus, Susie Essman, Mark...",English,6.897374
The Secret Life of Pets (2016.0),PG,The quiet life of a terrier named Max is upend...,2016,"Animation, Adventure, Comedy, Family","Chris Renaud, Yarrow Cheney(co-director)","Louis C.K., Eric Stonestreet, Kevin Hart, Jenn...",English,6.498116
Trolls (2016),PG,"After the Bergens invade Troll Village, Poppy,...",2016,"Animation, Adventure, Comedy, Family, Fantasy,...","Mike Mitchell, Walt Dohrn(co-director)","Anna Kendrick, Justin Timberlake, Zooey Descha...",English,6.495212
Hoodwinked! (2005),PG,"Little Red Riding Hood, the Wolf, the Woodsman...",2005,"Animation, Comedy, Crime, Family, Musical, Mys...","Cory Edwards, Todd Edwards(co-director), Tony ...","Anne Hathaway, Glenn Close, Jim Belushi, Patri...","English, Spanish",6.494235
Planet 51 (2009),PG,An alien civilization is invaded by Astronaut ...,2009,"Animation, Adventure, Comedy, Family, Sci-Fi","Jorge Blanco, Javier Abad(co-director), Marcos...","Dwayne Johnson, Jessica Biel, Justin Long, Gar...","English, Spanish",6.096812


In [31]:
return_recs('The Lord of the Rings: The Fellowship of the Ring (2001)', 'weighted_rating', df = movies)

Here are the movie recommendations for The Lord of the Rings: The Fellowship of the Ring (2001): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
The Lord of the Rings: The Return of the King (2003),PG-13,Gandalf and Aragorn lead the World of Men agai...,2003,"Adventure, Drama, Fantasy",Peter Jackson,"Noel Appleby, Ali Astin, Sean Astin, David Aston","English, Quenya, Old English, Sindarin",8.899177
The Lord of the Rings: The Two Towers (2002),PG,While Frodo and Sam edge closer to Mordor with...,2002,"Adventure, Drama, Fantasy",Peter Jackson,"Bruce Allpress, Sean Astin, John Bach, Sala Baker","English, Sindarin, Old English",8.699152
Harry Potter and the Deathly Hallows: Part 2 (2011),PG-13,"Harry, Ron, and Hermione search for Voldemort'...",2011,"Adventure, Drama, Fantasy, Mystery",David Yates,"Ralph Fiennes, Michael Gambon, Alan Rickman, D...",English,8.098670
The Hobbit: An Unexpected Journey (2012),PG-13,"A reluctant Hobbit, Bilbo Baggins, sets out to...",2012,"Adventure, Family, Fantasy",Peter Jackson,"Ian McKellen, Martin Freeman, Richard Armitage...",English,7.898836
Thor (2011),PG-13,The powerful but arrogant god Thor (Chris Hems...,2011,"Action, Adventure, Fantasy, Sci-Fi",Kenneth Branagh,"Chris Hemsworth, Natalie Portman, Tom Hiddlest...",English,6.999272
What Dreams May Come (1998),PG-13,"Chris Nielsen dies in car crash one night, and...",1998,"Drama, Fantasy, Romance",Vincent Ward,"Robin Williams, Cuba Gooding Jr., Annabella Sc...",English,6.994813
The Lovely Bones (2009),PG-13,Centers on a young girl who has been murdered ...,2009,"Drama, Fantasy, Thriller",Peter Jackson,"Mark Wahlberg, Rachel Weisz, Susan Sarandon, S...",English,6.697392
The Long and Short of It (2003),NaN,A workman gets some unexpected help from two p...,2003,"Short, Drama",Sean Astin,"Andrew Lesnie, Praphaphorn 'Fon' Chansantor, P...",English,6.627800
The Price of Milk (2000),PG-13,New Zealand milk farmer Rob gives his lover Lu...,2000,"Comedy, Drama, Fantasy",Harry Sinclair,"Danielle Cormack, Karl Urban, Willa O'Neill, M...",English,6.586552


In [44]:
recommendations('Spirited Away (2001)')

['Ponyo (2008)',
 'My Neighbor Totoro (1988)',
 "Howl's Moving Castle (2004)",
 'Porco Rosso (1992)',
 'The Cat Returns (2002)',
 "Kiki's Delivery Service (1989)",
 "Kiki's Delivery Service (1989)",
 'Nausicaä of the Valley of the Wind (1984)',
 'Castle in the Sky (1986)',
 'My Neighbors the Yamadas (1999)',
 'Nutcracker Fantasy (1979)',
 "Harry Potter and the Sorcerer's Stone (2001)",
 'Mei and the Kitten Bus (2002)',
 'Onigamiden - Legend of the Millennium Dragon (2011)',
 'Pom Poko (1994)']

In [33]:
return_recs('Spirited Away (2001)', 'weighted_rating', df = movies)

Here are the movie recommendations for Spirited Away (2001): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Howl's Moving Castle (2004),PG,When an unconfident young woman is cursed with...,2004,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,"Chieko Baishô, Takuya Kimura, Akihiro Miwa, Ta...",Japanese,8.196626
My Neighbor Totoro (1988),G,When two girls move to the country to be near ...,1988,"Animation, Family, Fantasy",Hayao Miyazaki,"Noriko Hidaka, Chika Sakamoto, Shigesato Itoi,...",Japanese,8.196011
Castle in the Sky (1986),PG,A young boy and a girl with a magic crystal mu...,1986,"Animation, Adventure, Drama, Fantasy, Sci-Fi",Hayao Miyazaki,"Anna Paquin, James Van Der Beek, Cloris Leachm...",Japanese,8.092852
Porco Rosso (1992),PG,"In 1930s Italy, a veteran World War I pilot is...",1992,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"Shûichirô Moriyama, Tokiko Katô, Bunshi Katsur...",Japanese,7.786986
Ponyo (2008),G,A five-year-old boy develops a relationship wi...,2008,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,"Tomoko Yamaguchi, Kazushige Nagashima, Yûki Am...",Japanese,7.692918
Harry Potter and the Sorcerer's Stone (2001),PG,An orphaned boy enrolls in a school of wizardr...,2001,"Adventure, Family, Fantasy",Chris Columbus,"Richard Harris, Maggie Smith, Robbie Coltrane,...",English,7.598725
Mei and the Kitten Bus (2002),NaN,This is the story of Mei and her friend the ki...,2002,"Animation, Short, Comedy",Hayao Miyazaki,"Hayao Miyazaki, Chika Sakamoto",Japanese,7.492856
Pom Poko (1994),PG,A community of magical shape-shifting raccoon ...,1994,"Animation, Comedy, Drama, Family, Fantasy",Isao Takahata,"Shinchô Kokontei, Makoto Nonomura, Yuriko Ishi...","Japanese, English",7.367225
The Cat Returns (2002),G,"After helping a cat, a 17-year-old girl finds ...",2002,"Animation, Adventure, Comedy, Family, Fantasy,...",Hiroyuki Morita,"Chizuru Ikewaki, Yoshihiko Hakamada, Aki Maeda...","Japanese, English",7.284617


In [34]:
return_recs('Titanic (1997)', 'weighted_rating', df = movies)

Here are the movie recommendations for Titanic (1997): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
The Notebook (2004),PG-13,A poor yet passionate young man falls in love ...,2004,"Drama, Romance",Nick Cassavetes,"Tim Ivey, Gena Rowlands, Starletta DuPois, Jam...",English,7.898228
500 Days of Summer (2009),PG-13,An offbeat romantic comedy about a woman who d...,2009,"Comedy, Drama, Romance",Marc Webb,"Joseph Gordon-Levitt, Zooey Deschanel, Geoffre...","English, French, Swedish",7.698267
Ghosts of the Abyss (2003),G,"James Cameron and Bill Paxton, director and ac...",2003,"Documentary, Family",James Cameron,"Bill Paxton, John Broadwater, Lori Johnston, C...",English,6.810055
The Luzhin Defence (2000),PG-13,Two worlds collide when an eccentric genius fa...,2000,"Drama, Romance",Marleen Gorris,"John Turturro, Emily Watson, Geraldine James, ...","English, Italian",6.796942
Where the Heart Is (2000),PG-13,A pregnant 17-year-old rebuilds her life after...,2000,"Comedy, Drama, Romance",Matt Williams,"Natalie Portman, Ashley Judd, Stockard Channin...",English,6.786285
Crazy/Beautiful (2001),PG-13,"At Pacific Palisades High, a poor Latino falls...",2001,"Drama, Romance",John Stockwell,"Kirsten Dunst, Jay Hernandez, Bruce Davison, H...",English,6.486972
Dear John (2010),PG-13,A romantic drama about a soldier who falls for...,2010,"Drama, Romance, War",Lasse Hallström,"Channing Tatum, Amanda Seyfried, Richard Jenki...",English,6.298220
Stanley & Iris (1990),PG-13,A struggling widow falls in love with an illit...,1990,"Drama, Romance",Martin Ritt,"Jane Fonda, Robert De Niro, Swoosie Kurtz, Mar...",English,6.266530
Tadpole (2002),PG-13,Coming-of-age story about a suave 15-year-old ...,2002,"Comedy, Drama, Romance",Gary Winick,"Aaron Stanford, Kate Mara, Robert Iler, Peter ...","English, French",6.167841


In [36]:
return_recs('Wings of Desire (1987)', 'weighted_rating', movies)

Here are the movie recommendations for Wings of Desire (1987): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Alice in the Cities (1974),Not Rated,A German journalist is saddled with a nine-yea...,1974,Drama,Wim Wenders,"Rüdiger Vogler, Yella Rottländer, Lisa Kreuzer...","German, English, Dutch",7.901960
"Lacombe, Lucien (1974)",R,"In 1944, an 18-year old boy from small-town Fr...",1974,"Drama, Romance, War",Louis Malle,"Pierre Blaise, Aurore Clément, Holger Löwenadl...","French, German, English",7.667056
Pina (2011),PG,"A tribute to the late German choreographer, Pi...",2011,"Documentary, Music",Wim Wenders,"Regina Advento, Malou Airaudo, Ruth Amarante, ...","German, French, English, Spanish, Croatian, It...",7.644937
The American Friend (1977),Not Rated,"Tom Ripley, who deals in forged art, suggests ...",1977,"Crime, Drama, Mystery, Thriller",Wim Wenders,"Dennis Hopper, Bruno Ganz, Lisa Kreuzer, Gérar...","German, English, French",7.346569
"Faraway, So Close! (1993)",PG-13,A group of angels in the German capital look l...,1993,"Drama, Fantasy, Romance",Wim Wenders,"Otto Sander, Peter Falk, Horst Buchholz, Mikha...","German, French, English, Italian, Russian",7.214520
Silent Light (2007),NaN,"In a Mennonite community in Mexico, a father's...",2007,"Drama, Romance",Carlos Reygadas,"Cornelio Wall, Miriam Toews, Maria Pankratz, P...","Low German, Spanish, French, English",7.190581
Tokyo-Ga (1985),NaN,"Moved by the work of director Yasujirô Ozu, Wi...",1985,Documentary,Wim Wenders,"Chishû Ryû, Werner Herzog, Yûharu Atsuta","English, Japanese, German",7.143067
Land of Plenty (2004),NOT RATED,A drama that investigates anxiety and disillus...,2004,"Drama, War",Wim Wenders,"Michelle Williams, John Diehl, Shaun Toub, Wen...",English,6.422802
Room 666 (1982),NaN,During the '35th Cannes International Film Fes...,1982,Documentary,Wim Wenders,"Wim Wenders, Michelangelo Antonioni, Maroun Ba...","English, French, German, Italian, Turkish, Por...",6.258786


In [38]:
return_recs('Amélie (2001)', 'weighted_rating', movies)

Here are the movie recommendations for Amélie (2001): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Stolen Kisses (1968),R,"After being discharged from the army, Antoine ...",1968,"Comedy, Drama, Romance",François Truffaut,"Jean-Pierre Léaud, Delphine Seyrig, Claude Jad...","French, English",7.725378
Delicatessen (1991),R,Post-apocalyptic surrealist black comedy about...,1991,"Comedy, Crime","Marc Caro, Jean-Pierre Jeunet","Pascal Benezech, Dominique Pinon, Marie-Laure ...",French,7.689895
A Little Romance (1979),PG,A French boy (Daniel) and an American girl (La...,1979,"Adventure, Comedy, Romance",George Roy Hill,"Laurence Olivier, Diane Lane, Thelonious Berna...","English, French, Italian",7.366602
Rendezvous in Paris (1995),NaN,Three stories of love and coincidence around t...,1995,"Comedy, Romance",Éric Rohmer,"Clara Bellar, Antoine Basler, Mathias Mégard, ...",French,7.114253
Romantics Anonymous (2010),PG-13,What happens when a man and a woman share a co...,2010,"Comedy, Romance",Jean-Pierre Améris,"Benoît Poelvoorde, Isabelle Carré, Lorella Cra...",French,6.855191
Brief Crossing (2001),Unrated,A young French man and an older English woman ...,2001,"Comedy, Drama, Romance",Catherine Breillat,"Sarah Pratt, Gilles Guillain, Marc Filipi, Laë...","French, English",6.653817
27 Missing Kisses (2000),NaN,"The summerly adventures of teen Sybilla, who f...",2000,"Comedy, Drama, Romance",Nana Dzhordzhadze,"Nutsa Kukhianidze, Evgeniy Sidikhin, Shalva Ia...","Georgian, Russian, French, English",6.547627
Beautiful Lies (2010),NaN,A hairdresser forwards a passionate love lette...,2010,"Comedy, Romance",Pierre Salvadori,"Audrey Tautou, Nathalie Baye, Sami Bouajila, S...","French, Chinese, English",6.456519
Mr. Bean's Holiday (2007),G,Mr. Bean wins a trip to Cannes where he unwitt...,2007,"Comedy, Family",Steve Bendelack,"Rowan Atkinson, Steve Pemberton, Lily Atkinson...","English, French, Spanish, Russian",6.397489


In [40]:
return_recs("Baby Driver (2017)", 'weighted_rating', movies)

Here are the movie recommendations for Baby Driver (2017): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Straw Dogs (1971),R,A young American and his English wife come to ...,1971,"Crime, Drama, Thriller",Sam Peckinpah,"Dustin Hoffman, Susan George, Peter Vaughan, T...",English,7.486885
Mr. Holland's Opus (1995),PG,A frustrated composer finds fulfillment as a h...,1995,"Drama, Music",Stephen Herek,"Richard Dreyfuss, Glenne Headly, Jay Thomas, O...","English, American Sign Language",7.281932
Absolute Power (1997),R,A career thief witnesses a horrific crime invo...,1997,"Action, Crime, Drama, Thriller",Clint Eastwood,"Clint Eastwood, Gene Hackman, Ed Harris, Laura...","English, Spanish",6.691833
New Jack City (1991),R,A crime lord ascends to power and becomes mega...,1991,"Action, Crime, Drama, Thriller",Mario Van Peebles,"Wesley Snipes, Ice-T, Allen Payne, Chris Rock",English,6.588204
Heist (2001),R,A career jewel thief finds himself at tense od...,2001,"Action, Crime, Drama, Thriller",David Mamet,"Gene Hackman, Danny DeVito, Delroy Lindo, Sam ...",English,6.490304
Q & A (1990),R,A young district attorney seeking to prove a c...,1990,"Crime, Drama, Thriller",Sidney Lumet,"Nick Nolte, Timothy Hutton, Armand Assante, Pa...","English, Spanish",6.440881
The Principal (1987),R,A teacher is assigned to be the principal of a...,1987,"Action, Crime, Drama, Thriller",Christopher Cain,"Jim Belushi, Louis Gossett Jr., Rae Dawn Chong...",English,6.273172
Wrong Turn at Tahoe (2009),R,A small-time crime boss kills a drug dealer wi...,2009,"Action, Crime, Drama, Thriller",Franck Khalfoun,"Cuba Gooding Jr., Miguel Ferrer, Harvey Keitel...",English,6.062140
The Crew (2008),R,As a major heist approaches and with betrayal ...,2008,"Action, Crime, Drama, Thriller",Adrian Vitoria,"Stephen Graham, Scot Williams, Rory McCann, Ke...",English,5.960632


In [42]:
return_recs("Spirited Away (2001)", 'weighted_rating', movies)

Here are the movie recommendations for Spirited Away (2001): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Howl's Moving Castle (2004),PG,When an unconfident young woman is cursed with...,2004,"Animation, Adventure, Family, Fantasy",Hayao Miyazaki,"Chieko Baishô, Takuya Kimura, Akihiro Miwa, Ta...",Japanese,8.196626
My Neighbor Totoro (1988),G,When two girls move to the country to be near ...,1988,"Animation, Family, Fantasy",Hayao Miyazaki,"Noriko Hidaka, Chika Sakamoto, Shigesato Itoi,...",Japanese,8.196011
Castle in the Sky (1986),PG,A young boy and a girl with a magic crystal mu...,1986,"Animation, Adventure, Drama, Fantasy, Sci-Fi",Hayao Miyazaki,"Anna Paquin, James Van Der Beek, Cloris Leachm...",Japanese,8.092852
Porco Rosso (1992),PG,"In 1930s Italy, a veteran World War I pilot is...",1992,"Animation, Adventure, Comedy, Fantasy, Romance",Hayao Miyazaki,"Shûichirô Moriyama, Tokiko Katô, Bunshi Katsur...",Japanese,7.786986
Ponyo (2008),G,A five-year-old boy develops a relationship wi...,2008,"Animation, Adventure, Comedy, Family, Fantasy",Hayao Miyazaki,"Tomoko Yamaguchi, Kazushige Nagashima, Yûki Am...",Japanese,7.692918
Harry Potter and the Sorcerer's Stone (2001),PG,An orphaned boy enrolls in a school of wizardr...,2001,"Adventure, Family, Fantasy",Chris Columbus,"Richard Harris, Maggie Smith, Robbie Coltrane,...",English,7.598725
Mei and the Kitten Bus (2002),NaN,This is the story of Mei and her friend the ki...,2002,"Animation, Short, Comedy",Hayao Miyazaki,"Hayao Miyazaki, Chika Sakamoto",Japanese,7.492856
Pom Poko (1994),PG,A community of magical shape-shifting raccoon ...,1994,"Animation, Comedy, Drama, Family, Fantasy",Isao Takahata,"Shinchô Kokontei, Makoto Nonomura, Yuriko Ishi...","Japanese, English",7.367225
The Cat Returns (2002),G,"After helping a cat, a 17-year-old girl finds ...",2002,"Animation, Adventure, Comedy, Family, Fantasy,...",Hiroyuki Morita,"Chizuru Ikewaki, Yoshihiko Hakamada, Aki Maeda...","Japanese, English",7.284617


In [46]:
return_recs("Alice in Wonderland (2010)", 'weighted_rating', movies)

Here are the movie recommendations for Alice in Wonderland (2010): 


,rated,plot,year,genre,director,actors,language,weighted_rating
title_year,,,,,,,,
Harry Potter and the Sorcerer's Stone (2001),PG,An orphaned boy enrolls in a school of wizardr...,2001,"Adventure, Family, Fantasy",Chris Columbus,"Richard Harris, Maggie Smith, Robbie Coltrane,...",English,7.598725
Alice in Wonderland (1951),G,Alice stumbles into the world of Wonderland. W...,1951,"Animation, Adventure, Family, Fantasy, Musical","Clyde Geronimi, Wilfred Jackson, Hamilton Luske","Kathryn Beaumont, Ed Wynn, Richard Haydn, Ster...",English,7.394441
Mirrormask (2005),PG,"In a fantasy world of opposing kingdoms, a fif...",2005,"Adventure, Fantasy",Dave McKean,"Stephanie Leonidas, Jason Barry, Rob Brydon, G...",English,6.879333
Charlie and the Chocolate Factory (2005),PG,A young boy wins a tour through the most magni...,2005,"Adventure, Comedy, Family, Fantasy, Musical",Tim Burton,"Johnny Depp, Freddie Highmore, David Kelly, He...",English,6.599121
Alice in Wonderland (2010),PG,Nineteen-year-old Alice returns to the magical...,2010,"Adventure, Family, Fantasy",Tim Burton,"Johnny Depp, Mia Wasikowska, Helena Bonham Car...",English,6.499150
Zathura: A Space Adventure (2005),PG,Two young brothers are drawn into an intergala...,2005,"Action, Adventure, Comedy, Family, Fantasy, Sc...",Jon Favreau,"Jonah Bobo, Josh Hutcherson, Dax Shepard, Kris...",English,6.197735
Alice Through the Looking Glass (2016.0),PG,Alice returns to the whimsical world of Wonder...,2016,"Adventure, Family, Fantasy",James Bobin,"Johnny Depp, Mia Wasikowska, Helena Bonham Car...",English,6.197608
Little Monsters (1989),PG,A boy discovers an incredible and gruesome wor...,1989,"Adventure, Comedy, Family, Fantasy",Richard Greenberg,"Fred Savage, Howie Mandel, Daniel Stern, Marga...",English,6.085298
Percy Jackson & the Olympians: The Lightning Thief (2010),PG,A teenager discovers he's the descendant of a ...,2010,"Adventure, Family, Fantasy",Chris Columbus,"Logan Lerman, Brandon T. Jackson, Alexandra Da...","English, Greek, Ancient (to 1453)",5.899538


# Key Findings

1. This is a good initial filter for a website or app that gives movie recommendations for movies that are similar
2. Depending on the movie, key metrics like diveristy/variety differ. Below is analysis for a few of the recommendations:

## 1. The Lord of the Rings: The Fellowship of the Ring
All the movies that are also in the Lord of the Rings Franchise are returned, along with Harry Potter. But it also suggests a variety of other movies that are somewhat similar in tone, though none of them seem to be High Fantasy the same way that Lord of the Rings is. It's important to note, however, that there are two filters: The first filters for cosine similarity, the second filters the results by the weighted IMDB rating. By cosine similarity **only**, we also get movies from the Hobbit franchise, which are closely related to Lord of the Rings as it exists in the same Universe. However, these movies had a lower weighted rating on IMDB than other movies, so were suggested later. 

Movies recommended **ordered by cosine similarity**:

1. **'The Lord of the Rings: The Return of the King'**, also in the franchise
2. **'The Lord of the Rings: The Two Towers'**, also in the franchise
3. **'The Lord of the Rings (1978)**, the 1978 animated film of the same movie
4. **'Harry Potter and the Deathly Hallows: Part 2'** a similar movie in terms of high fantasy and plot
5. **'The Hobbit: The Desolation of Smaug'** also in the franchise
6. **'The Hobbit: An Unexpected Journey'** also in the franchise
7. **'The Long and Short of It'** - a short film made by crew during the filming of LoTR: The Two Towers
8. **'What Dreams May Come'** - a 1998 film that features tangential similarities, a 'journey' through a foreign land as the main character traverses through the afterlife in various forms, I'm not sure if this would be popular for lovers of LoTR as it's a different kind of 'fantasy' and 'drama'
9. **'The Hobbit: The Battle of the Five Armies'** another Hobbit film
10. **'The Lovely Bones'** - Another movie that involves the afterlife, I'm not sure this would be popular among people who love LoTR
11. **'The Price of Milk'** - A romantic comedy filmed in New Zealand, where the LoTR franchise was filmed
12. **'Hunt for the Wilderpeople'** - Another example of I think the algorithm using 'drama'
13. **'Thor'** - probably more of a movie that would be enjoyed by LoTR fans, as it's fantasy and deals more with 'war'
14. **'Captain Marvel'** - Similar to Thor, I think this is closer to what users would want to be recommended
15. **'Camille'** - has to do with journey through the afterlife


### Overall takeaway for LoTR: Fellowship Recommendations

The recommendations all have to do with a hero's journey, loss, and in many cases the afterlife. Most of them have some sort of fantastical element, though the recommendations fall short of what LoTR fans might like. I don't think this is a great filter to show to people who love recommendations, but think it could be improved if there were some more keywords (perhaps user-inputted keywords like 'High fantasy' and 'war'). 

The MovieLens recommender also recommends Star Wars and Indiana Jones based on Lord of the Rings. For computational time, this version of the filter only uses movies from 1988 and beyond, and therefore misses Star Wars and Indiana Jones, for the most part, but not the new Star Wars movies, which I suppose are not close enough to be recommended.

It's important to note that, in terms of themes, these **are good picks**. These movies all have thematic links, about loss and sacrifice and the inevitability of entropy and how power attracts the corruptible. Except Joe Dirt, I'm not sure what that's about.



## 2. Spirited Away
The results for *Spirited Away*, the award-winning 2001 animation, are a little more consistent across the board. Most of the other movies are also Japanese animation from the same studio, Studio Ghibli, making the diversity and variety of this set lower than Lord of the Rings, which recommended quite a few different genres, directors, and film ideas (which is good and bad, in its own way).

Movies recommended **filtered by cosine similarity and ordered by the weighted IMDB rating**:

* **Howl's Moving Castle** - deals with many similar themes and is from the same studio/director
* **My Neighbor Totoro** - also from Miyazaki and Studio Ghibli, this movie also features strong female leads, as two girls who have moved to a new home (similar) encounter strange creatures and beasts
* **Castle in the Sky** - Another Miyazaki with similar qualities, female leads, strange journies and beasts (to be fair, these themes run through most of Miyazaki's works)
* **Porco Rosso** - Also from Miyazaki
* **Ponyo** - Also from Miyazaki
* **Harry Potter and the Sorcerer's Stone** - a similar theme and idea of a child taken away from their parents and treated poorly by those around them (Harry loses his parents, Chihiro in Spirited Away has her parents turned into pigs). Other, magical themes run through both of these stories, making this a decent recommendation.
* **Mei and the Kitten Bus** - a spinoff short film with characters from Miyazaki's My Neighbor Totoro, this takes the younger girl Mei and the 'kitten bus' (based off the cat bus) from that film.
* **Pom Poko** - from Studio Ghibli / Miyazaki
* **The Cat Returns** - from Studio Ghibli / Miyazaki
* **My Neighbors the Yamadas** - from Studio Ghibli / Miyazaki


### Overall takeway for Spirited Away:
These recommendations are certainly more consistent and better than they were for Lord of Rings, but are a little strict in terms of who the director is, for example. I would have loved to get back some other Japanese animation (however I don't even know if that's in my dataframe). However, again, the algorithm has returned movies that rest on pretty similar thematic lines.

## Overall Findings

The algorithm is good at finding underlying thematic tones of movies and returning them, even if they aren't strictly in the same genre. Some tuning to the algorithm would perhaps enhance it, playing with NLP to see what could be done with n-grams instead of just a bag of words, or other processes like word2vec, as well as lemmatization or stemming, which I did not do in this iteration of the filter. Overall, I'm happy with how it performs, and thin it would be a good initial interface for someone to interact with.

## Ideas for further tuning and research

I'd love to try this out with more tags. For example, *Spirited Away*'s original language was Japanese, but it was translated and widely released into English, and Studio Ghibli is partnered with Pixar, who is owned by Disney. These lines could help bridge the gaps between certain movies. In an earlier version of this filter, Spirited Away returned *Alice in Wonderland*, which thrilled me. That doesn't seem to be the case this time, which is disappointing, but makes me wonder about how we might get a machine to see those gaps and leap across them, especially using a distance function such as cosine similarity. 